# Исследование надёжности заёмщиков

**Цель исследования:**
- Определить, влияет ли семейное положение и количество детей клиента банка на факт погашения им кредита в срок.

**Входные данные от банка:**
- Статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Необходимо ответить на следующие вопросы:**
- Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

**Описание данных (источник - кредитный отдел банка):**
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

## 1. Обзор данных

In [58]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [59]:
bank = pd.read_csv('/Users/yusuf/Downloads/data (1).csv')
bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Сумма данных по количеству проработанных дней (days_employed) и общему заработку (total_income) совпадают, при этом отстают от других показателей на 2174 вывода - стоит обратить на это внимание, скорее всего в первом случае - это официально не работавшие люди без заработка.

In [60]:
bank.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


- Количество проработанных дней должно быть целым, положительным числом 
- Также есть необходимость привести в порядок регистры в столбце с образованием

## 2. Предобработка данных

### 2.1 Обработка пропусков

Заменим вышеперечисленные пропуски на медианные значения по типу дохода, так как среднее может быть сильно искажено отдельными "высокими" или "низкими" показателями

In [61]:
bank['total_income'].fillna(value = bank.groupby(['income_type'])['total_income'].transform('median'), inplace = True)
bank['days_employed'].fillna(value = bank.groupby(['income_type'])['days_employed'].transform('median'), inplace = True)

In [62]:
bank.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Теперь пропуска заменены. Скорее всего их не указали в анкетах одни и те же люди по причине отсуствия официальной работы и, соответственно официального заработка. 

### 2.2 Замена типа данных

Приведём количество проработанных дней к положительному и целому значению; 
сделаем коррекцию правописания в столбце "образование";
приведем к целому показатель "общий доход"

In [63]:
bank['days_employed'] = bank['days_employed'].abs()
bank['total_income'] = bank['total_income'].astype(int) 
bank['days_employed'] = bank['days_employed'].astype(int) 
bank['education'] = bank['education'].str.lower()

In [64]:
bank.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


In [65]:
bank['children'] = bank['children'].abs() #это сделано по причине нахождения значения "-1", что скорее всего является опечаткой при внесении данных
bank['children'] = bank['children'].replace(20, 2) #вероятнее всего "20" - это опечатка, и подразумевается значение "2"

### 2.3 Обработка дубликатов

In [66]:
bank.duplicated().sum()

71

In [67]:
bank = bank.drop_duplicates().reset_index(drop=True)
bank.duplicated().sum()

0

Все дубликаты были почищенны

### 2.4 Лемматизация

In [68]:
m = Mystem()
bank['lemmas'] = bank['purpose'].apply(m.lemmatize)

In [69]:
print(Counter(bank['lemmas'].sum()))

Counter({' ': 33570, '\n': 21454, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


С помощью лемматизации были выделены слова из столбца "назначение кредита", заметно, что лидирует "недвижимость", при этом повторяясь с "жильем".

### 2.5 Категоризация данных

In [70]:
#Разделю приобретателей кредита по уровням дохода

def income_group(total_income):
    if 0 < total_income < 30000:
        return 'E'
    if 30001 < total_income < 50000:
        return 'D'
    if 50001 < total_income < 200000:
        return 'C'
    if 200001 < total_income < 1000000:
        return 'B'
    if total_income > 1000001:
        return 'A'

bank['income_group'] = bank['total_income'].apply(income_group)

In [71]:
#Разделю похожие цели кредита на основные группы

def purpose_lemmas(purpose):
    if 'недвижимость' in purpose or 'жилье' in purpose:
        return 'операции с недвижимостью'
    if 'автомобиль' in purpose:
        return 'операции с автомобилем'
    if 'образование' in purpose:
        return 'получение образования'
    if 'свадьба' in purpose:
        return 'проведение свадьбы'
    return 'Прочее'

bank['purpose_main'] = bank['lemmas'].apply(purpose_lemmas)

In [72]:
bank['purpose_main'].value_counts()

операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_main, dtype: int64

Главная цель кредита - операции с недвижимостю. В 2 раза меньше кредитов берут на операции с автомобилем и получение образования

In [73]:
bank['income_group'].value_counts()

C    16015
B     5041
D      350
A       25
E       22
Name: income_group, dtype: int64

Наибольшее количество замещиков имеют достаток от 50 до 200 тыс. рублей. Для операций с недвижимостью необходим доход выше среднего 

## 3. Ответы на поставленные вопросы

### 3.1 Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [74]:
children_pivot = bank.pivot_table(index = 'children', columns = 'debt', values = 'income_type', aggfunc = 'count')
children_pivot['share'] = children_pivot[1]/(children_pivot[0] + children_pivot[1])

for category in depend_pivot.index:
    print('Процент в категории {}, которые не вернули кредит в срок {:.1%}'.format(category, children_pivot.loc[category, 'share']))

Процент в категории 0, которые не вернули кредит в срок 7.5%
Процент в категории 1, которые не вернули кредит в срок 9.2%
Процент в категории 2, которые не вернули кредит в срок 9.5%
Процент в категории 3, которые не вернули кредит в срок 8.2%
Процент в категории 4, которые не вернули кредит в срок 9.8%
Процент в категории 5, которые не вернули кредит в срок nan%


Бездетные кредиторы следят за графиком платежей внимательнее всего. При появлении первого ребёнка этот показатель ухудшается, достигая наибольшего значения с 4 детьми, хотя различия с 1 ребенком в семье минимальны (6 б.п.). Скорее всего это связано с возраставющими обязательствами при появлении детей

### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [75]:
family_pivot = bank.pivot_table(index = 'family_status', columns = 'debt', values = 'income_type', aggfunc = 'count')
family_pivot['share'] = family_pivot[1]/(family_pivot[0] + family_pivot[1])

for category in family_pivot.index:
    print('Процент в категории {}, которые не вернули кредит в срок {:.1%}'.format(category, family_pivot.loc[category, 'share']))

Процент в категории Не женат / не замужем, которые не вернули кредит в срок 9.8%
Процент в категории в разводе, которые не вернули кредит в срок 7.1%
Процент в категории вдовец / вдова, которые не вернули кредит в срок 6.6%
Процент в категории гражданский брак, которые не вернули кредит в срок 9.3%
Процент в категории женат / замужем, которые не вернули кредит в срок 7.5%


Наибольший процент невозврата имеют холостые - возможно им некому напомнить о кредите, либо некому "подстраховать" при нехватке. Далее идут люди состоящие в гражданском браке, скорее всего это молодые люди с низким доходом, берущие кредит "на свадьбу", окрыленные перспективами совместной жизни. Люди в разводе и замужние/женатые возвращают одинакого - скорее всего это люди обладающие стабильным уровнем дохода. Люди потерявшие свои вторую половину обладают самым низким процентом невозврата

### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [76]:
income_pivot = bank.pivot_table(index = 'income_group', columns = 'debt', values = 'income_type', aggfunc = 'count')
income_pivot['share'] = income_pivot[1]/(income_pivot[0] + income_pivot[1])

for category in income_pivot.index:
    print('Процент в категории {}, которые не вернули кредит в срок {:.1%}'.format(category, income_pivot.loc[category, 'share']))

Процент в категории A, которые не вернули кредит в срок 8.0%
Процент в категории B, которые не вернули кредит в срок 7.1%
Процент в категории C, которые не вернули кредит в срок 8.5%
Процент в категории D, которые не вернули кредит в срок 6.0%
Процент в категории E, которые не вернули кредит в срок 9.1%


Категория с самым низким уровнем дозода обладает самым высоким показателем просрочки платежей. При этом самым низким уровнем просрочки обладает категория с доходом от 30 до 50 тыс. руб.

### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [77]:
purpose_pivot = bank.pivot_table(index = 'purpose_main', columns = 'debt', values = 'income_type', aggfunc = 'count')
purpose_pivot['share'] = purpose_pivot[1]/(purpose_pivot[0] + purpose_pivot[1])

for category in purpose_pivot.index:
    print('Процент в категории {}, которые не вернули кредит в срок {:.1%}'.format(category, purpose_pivot.loc[category, 'share']))

Процент в категории операции с автомобилем, которые не вернули кредит в срок 9.4%
Процент в категории операции с недвижимостью, которые не вернули кредит в срок 7.2%
Процент в категории получение образования, которые не вернули кредит в срок 9.2%
Процент в категории проведение свадьбы, которые не вернули кредит в срок 8.0%


По просрочкам лидируют категории "автомобиль" и "образование", что может быть связано с тем, что эти кредиты берут молодые люди, студенты, с менее крепким финансовым состоянием. При этом "недвижимость" возвратнее всех, это более осознанная покупка, к которой готовятся люди. Также это часто является инвестицией

## 4. Общий вывод

- **Краткий обзор проведённой работы.** В рамках предобработки данных были обработаны пропуска и дубликаты, изменены типы на корректные, проведена лематизация и категоризация. Были найдены ответы на все поставленные вопросы 
- **Главные выводы.** В рамках работы была найдена зависимость между возвратом кредита в срок и наличием детей, а также семейным положением, уровнем дохода и назначением кредита. Наиболее желанным с точки зрения возврата для банка являются бездетные вдовцы/вдовы, берущие кредит для операций с недвижимостью. Самыми нежеланными являются родители 4 детей либо неженатые/находящиеся в гражданском браке, берущие кредит для оплаты образования либо операций с автомобилем